<a href="https://colab.research.google.com/github/jphermans/vcg/blob/main/FilterToolsModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install xlrd

In [ ]:
import pandas as pd
from google.colab import files
import io
import os
import numpy as np

# --- Configuratie voor het markeren ---
KLEUR_TEKST_DUPLICATEN_KOLOM1 = 'red' # Rode tekst voor duplicaten in kolom 1
KLEUREN_ZEBRA_RIJEN = ['#f0f0f0', '#e6f3ff'] # Lichtgrijs, Lichtblauw

def stijl_duplicaten_kolom1_en_zebra(df_te_stijlen, originele_bestandsnaam):
    """
    Stijlt rijen:
    - Identificeert rijen met duplicate waarden in de EERSTE kolom.
    - Kleurt tekst van deze rijen rood.
    - Geeft niet-gemarkeerde rijen afwisselende achtergrondkleuren (zebra).
    - Formatteert kolom 4 & 5 (indien numeriek en aanwezig) voor weergave op 2 decimalen.
    Geeft een Styler object terug.
    """
    if df_te_stijlen.empty:
        print("DataFrame is leeg, geen styling toe te passen.")
        return df_te_stijlen.style

    # We hebben geen aparte 'df_voor_logica' meer nodig voor complexe afronding VOOR duplicaatcheck,
    # omdat de duplicaatcheck nu alleen op de eerste kolom is.
    # Afronding voor weergave gebeurt later met .format().

    # --- Stap 1: Identificeren van duplicaten in de EERSTE kolom ---
    is_duplicaat_kolom1_rij = pd.Series(False, index=df_te_stijlen.index)
    naam_eerste_kolom = ""

    if df_te_stijlen.shape[1] > 0: # Zorg ervoor dat er minstens één kolom is
        naam_eerste_kolom = df_te_stijlen.columns[0]
        # .duplicated(keep=False) markeert ALLE voorkomens van duplicaten als True
        duplicaat_masker_kolom1 = df_te_stijlen.duplicated(subset=[naam_eerste_kolom], keep=False)

        if duplicaat_masker_kolom1.any():
            is_duplicaat_kolom1_rij[duplicaat_masker_kolom1] = True
            print(f"Duplicaten gevonden in eerste kolom ('{naam_eerste_kolom}'). Rijen worden rood gemarkeerd.")
        else:
            print(f"Geen duplicaten gevonden in eerste kolom ('{naam_eerste_kolom}').")
    else:
        print("DataFrame heeft geen kolommen, duplicaat check overgeslagen.")


    # --- Stap 2: Styling Functie (kleur tekst en zebra) ---
    zebra_teller = 0
    def pas_styling_toe(rij_series_voor_stijl):
        nonlocal zebra_teller
        # Gebruik de index van de rij om de duplicaatstatus op te zoeken
        is_duplicaat_in_kol1 = is_duplicaat_kolom1_rij.get(rij_series_voor_stijl.name, False)

        stijlen_rij = ['' for _ in rij_series_voor_stijl]
        if is_duplicaat_in_kol1:
            # Pas rode tekstkleur toe
            for i in range(len(stijlen_rij)):
                stijlen_rij[i] += f'color: {KLEUR_TEKST_DUPLICATEN_KOLOM1};'
        else:
            # Pas zebra achtergrondkleur toe
            achtergrondkleur = KLEUREN_ZEBRA_RIJEN[zebra_teller % len(KLEUREN_ZEBRA_RIJEN)]
            for i in range(len(stijlen_rij)):
                stijlen_rij[i] += f'background-color: {achtergrondkleur};'
            zebra_teller += 1
        return stijlen_rij

    # Start styling met de df_te_stijlen (de geselecteerde 5 kolommen)
    gestijlde_df = df_te_stijlen.style.apply(pas_styling_toe, axis=1)

    # --- Stap 3: Formatteren voor WEERGAVE (2 decimalen voor kolom 4 & 5) ---
    format_dict = {}
    kolom_namen_df = df_te_stijlen.columns

    # Kolom 4 (index 3)
    if len(kolom_namen_df) > 3:
        # Probeer te formatteren als de kolom potentieel numeriek kan zijn.
        # We converteren hier niet meer, vertrouwen op Styler.format's vermogen om met types om te gaan.
        # Als een waarde niet naar float met 2 decimalen geformatteerd kan worden, blijft het origineel.
        try:
            # Test of de kolom numeriek kan zijn door een conversie te proberen op een kopie
            if pd.api.types.is_numeric_dtype(pd.to_numeric(df_te_stijlen[kolom_namen_df[3]], errors='coerce')):
                 format_dict[kolom_namen_df[3]] = "{:.2f}"
        except Exception:
            pass # Negeer als conversie voor test mislukt, formatteer dan niet

    # Kolom 5 (index 4)
    if len(kolom_namen_df) > 4:
        try:
            if pd.api.types.is_numeric_dtype(pd.to_numeric(df_te_stijlen[kolom_namen_df[4]], errors='coerce')):
                format_dict[kolom_namen_df[4]] = "{:.2f}"
        except Exception:
            pass

    if format_dict:
        print(f"\nFormatteren voor weergave (2 decimalen) op kolommen: {list(format_dict.keys())}")
        gestijlde_df = gestijlde_df.format(format_dict, na_rep='N/A', precision=2)
    else:
        print("\nGeen (numerieke) kolommen 4 en/of 5 gevonden om te formatteren voor 2-decimale weergave.")


    gestijlde_df = gestijlde_df.set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]},
        {'selector': 'td, th', 'props': [('border', '1px solid #dddddd'), ('padding', '8px')]},
        {'selector': 'tr:hover td', 'props': [('background-color', '#cce5ff !important')]}
    ]).set_caption(f"Tabel uit '{originele_bestandsnaam}' (Duplicaten Kolom 1 Rood) en Zebra Rijen")

    # Samenvatting output
    if is_duplicaat_kolom1_rij.any():
        print(f"'{KLEUR_TEKST_DUPLICATEN_KOLOM1}' tekstkleur toegepast op rijen met duplicaten in de eerste kolom.")
    print(f"Afwisselende rijkleuren toegepast op niet-gemarkeerde rijen: {KLEUREN_ZEBRA_RIJEN[0]} en {KLEUREN_ZEBRA_RIJEN[1]}.")

    return gestijlde_df


# --- Hoofd Blok voor Verwerking ---
print("Upload alstublieft uw Excel-bestand (.xlsx of .xls)")
geupload = files.upload()

if not geupload:
  print("Geen bestand geüpload.")
else:
  bestandsnaam = list(geupload.keys())[0]
  print(f"\nGeüpload bestand: '{bestandsnaam}'")

  try:
    bestandsinhoud = geupload[bestandsnaam]
    if bestandsnaam.endswith('.xlsx'):
      df_volledig = pd.read_excel(io.BytesIO(bestandsinhoud), engine='openpyxl')
    elif bestandsnaam.endswith('.xls'):
      df_volledig = pd.read_excel(io.BytesIO(bestandsinhoud), engine='xlrd')
    else:
      print("Niet-ondersteund bestandsformaat.")
      df_volledig = None

    if df_volledig is not None:
      # --- Selecteer de eerste 5 kolommen ---
      # Dit is de df die we daadwerkelijk gaan stylen en tonen (df_te_stijlen in de functie)
      if df_volledig.shape[1] == 0:
          print("\nHet geüploade Excel-bestand lijkt leeg te zijn of heeft geen kolommen.")
          df_geselecteerd_voor_weergave = pd.DataFrame()
      elif df_volledig.shape[1] < 5:
        print(f"\nHet bestand heeft slechts {df_volledig.shape[1]} kolom(men). Alle beschikbare kolommen worden gebruikt.")
        df_geselecteerd_voor_weergave = df_volledig.copy()
      else:
        df_geselecteerd_voor_weergave = df_volledig.iloc[:, 0:5].copy()
        print(f"\n--- De eerste 5 kolommen worden verwerkt en getoond: {list(df_geselecteerd_voor_weergave.columns)} ---")

      if not df_geselecteerd_voor_weergave.empty:
        gestijlde_output = stijl_duplicaten_kolom1_en_zebra(df_geselecteerd_voor_weergave, bestandsnaam)

        print("\n--- Gestijlde Tabel Output: ---")
        display(gestijlde_output)

        html_bestandsnaam = f"{os.path.splitext(bestandsnaam)[0]}_kolom1_dups_rood_zebra.html"
        try:
            with open(html_bestandsnaam, 'w', encoding='utf-8') as f:
                f.write(gestijlde_output.to_html(escape=False))
            print(f"\nGestijlde tabel opgeslagen als '{html_bestandsnaam}'. Download zal starten.")
            files.download(html_bestandsnaam)
        except Exception as e_html:
            print(f"Fout bij opslaan of downloaden van HTML: {e_html}")
      else:
          print("\nGeen data om te verwerken of weer te geven na kolomselectie.")

  except Exception as e:
    print(f"Er is een fout opgetreden: {e}")
    if '.xls' in bestandsnaam and 'xlrd' in str(e).lower():
        print("Voor .xls bestanden, overweeg xlrd te installeren: !pip install xlrd")